In [1]:
import shutil
import os
import re

In [2]:
thchs30='/home/godlovefei/CRHTK/thchs30'

In [3]:
def get_suffix(path,suffix): 
    txtfilenames=[]
    for dirpath,dirnames,filenames in os.walk(path): 
        filenames=filter(lambda filename:filename[-4:]==suffix,filenames) 
        filenames=map(lambda filename:os.path.join(dirpath,filename),filenames) 
        txtfilenames.extend(filenames)       
    return txtfilenames

In [4]:
def remove_sign(s,sign):#\d,.!? \\~\/:;@$%=+"
    return re.sub(r'(['+sign+'])','',s)

In [5]:
def utf_to_str(utf):
    return repr(utf).replace('u\'','').replace('\'','').replace('"','').replace(r'\\u','w').replace(r'\u','w')

In [6]:
def prepare_basic_data(thchs30):
    # prepare data prompts & train label
    for i in ['train','test']:
        data_src = thchs30 + '/data_thchs30/' + i
        #print(data_src)
        path_list = get_suffix('.'+'/thchs30/data_thchs30/'+ i,'.trn')
        #print(path_list)
        fw = open('data/' + i + '/'+i+'prompts','a')
        for path in path_list:
            print(path)
            lab_src = data_src + '/' + open(path).read()
            print(open(path).read())
            wrd_scp = repr(open(lab_src[:-1],'rb').read().decode('utf-8'))
            wrd_scp = utf_to_str(wrd_scp)
            wrd_scp = wrd_scp.split(r'\\n')
            Chinese = wrd_scp[0].replace('\\u','w').replace('l = ','')
            pinyin = wrd_scp[1]
            phones = remove_sign(wrd_scp[2],'\d')
            if i == 'train':
                wrd_scp = path.replace('.wav.trn','') + ' ' + Chinese + '\n'
                #print(wrd_scp)
                fw.write(wrd_scp)
                open(path.replace('.wav.trn','.lab'),'w').write(Chinese.replace(' ','\n'))
            else:
                wrd_scp = path.split('/')[-1].replace('.wav.trn','') + ' ' + Chinese + '\n'
                fw.write(wrd_scp)
        fw.close()
    for i in ['train','test']:
        data_src = thchs30 + '/data_thchs30/' + i
        fw1=open('scpfile'+'/'+i+'mfcc.scp','w')
        fw2=open('scpfile/' + i + '.scp','w')
        wav_list = get_suffix(data_src,'.wav')
        #print (data_src)
        wav_str='\n'.join(wav_list)
        mfcc_path=wav_str.replace('.wav','.mfc').replace('\\','/')

        mp=map(list,zip(wav_str.split('\n'),mfcc_path.split('\n')))
        pmt_list=[' '.join(i) for i in mp]
        wav_mfcc='\n'.join(pmt_list).replace('\\','/')
        fw1.write(wav_mfcc)
        fw2.write(mfcc_path)

In [7]:
def prepare_train_data():
    # generating words prompts for train & test
    if not os.path.exists('labels') : os.mkdir('labels')
    if not os.path.exists('labels') : os.mkdir('labels')
    cmd1 = 'perl HTKTutorial/prompts2mlf labels/trainwords.mlf data/train/trainprompts'
    os.system(cmd1)
    cmd2 = 'perl HTKTutorial/prompts2mlf labels/testwords.mlf data/test/testprompts'
    os.system(cmd2)
    
    # generating wlist
    if not os.path.exists('list') : os.mkdir('list')
    cmd3 = 'perl HTKTutorial/prompts2wlist data/train/trainprompts list/wlist'
    os.system(cmd3)
    # generating phones list
    cmd4 = 'HDMan -m -w list/wlist -n phones/nosilmonophones -l log/dlog dict/dict1 dict/lexicon dict/lexicon1'
    os.system(cmd4)
    
    # 复制monophones   不考虑声调问题，只利用声母韵母建模
    phones = open('phones/nosilmonophones','r').read().split('\n')
    mono = remove_sign(phones[0],'\d') + '\n'
    for i in range(1,len(phones)-1):
        if remove_sign(phones[i],'\d') != remove_sign(phones[i-1],'\d'):
            mono = mono + remove_sign(phones[i],'\d') + '\n'
    open('phones/monophones0','w').write(mono + 'sil' + '\n')
    open('phones/monophones1','w').write(mono + 'sp\nsil' + '\n')
    
    # insert mkphones0.led
    fw1 = open('ledfile/mkphones0.led','w')
    print('EX\nIS sil sil\nDE sp',file = fw1)
    # print >>fw1,'EX\nIS sil sil\nDE sp'
    fw1.close()
    
    # generating file phones0.Mlf
    cmd5 = 'HLEd -l "*" -d dict/dict1 -i labels/trainphones0.mlf ledfile/mkphones0.led labels/trainwords.mlf'
    os.system(cmd5)

In [8]:
def feature_extraction(thsch30):
    # 配置wav_config文件
    if not os.path.exists('configure/wav_config') : os.mkdir('configure/wav_config')
    fw2 = open('configure/wav_config','w')
    print('SOURCEFORMAT=WAVEFORM',file=fw2)
    print('SOURCEFORMAT=WAV',file=fw2)
    print('TARGETKIND=MFCC_0_D_A',file=fw2)
    print('TARGETRATE=100000.0',file=fw2)
    print('SAVECOMPRESSED=T',file=fw2)
    print('USEHAMMING=T',file=fw2)
    print('WINDOWSIZE=250000.0',file=fw2)
    print('SAVEWITHCRC=T',file=fw2)
    print('PREEMCOEF=0.97',file=fw2)
    print('NUMCHANS=26',file=fw2)
    print('CEPLIFTER=22',file=fw2)
    print('NUMCEPS=12',file=fw2)
    fw2.close() 

In [9]:
def calculate_mean_variance():
    # 生成主目录生成proto文件
    fw5 = open('proto','w')
    print('~o <STREAMINFO> 1 39',file=fw5)
    print('<VECSIZE> 39<NULLD><MFCC_D_A_0><DIAGC>',file=fw5)
    print('~h "proto"',file=fw5)
    print('<BEGINHMM>',file=fw5)
    print('<NUMSTATES> 5',file=fw5)
    print('<STATE> 2',file=fw5)
    print('<MEAN> 39',file=fw5)
    print(' -1.012254e+001 -7.264226e+000 -5.370609e+000 -9.712223e+000 -8.047407e+000 -5.681805e+000 \
            -5.715530e+000 -8.866713e-001 -3.691204e+000 -1.679087e+000 -2.372116e+000 -1.992831e+000 5.435896e+001 \
            3.500786e-003 3.751630e-003 -3.257445e-004 8.666397e-004 4.723764e-003 -5.870885e-004 2.242961e-004 \
            -5.077728e-003 -4.136859e-003 -6.560616e-004 -4.315113e-003 -2.692402e-003 3.635559e-003 -9.592683e-005 \
            -2.103158e-004 1.351117e-004 -1.661481e-005 -3.384908e-004 -2.177467e-004 -1.438305e-004 1.636888e-004 \
            2.628795e-004 5.652815e-006 2.228666e-004 2.034386e-004 -1.619475e-005',file=fw5)
    print('<VARIANCE> 39',file=fw5)
    print(' 1.019786e+002 5.942492e+001 7.969102e+001 8.208926e+001 7.801923e+001 7.487221e+001 \
                7.644022e+001 6.621936e+001 6.555122e+001 4.695240e+001 4.796211e+001 3.524432e+001 1.175200e+002 \
                4.562054e+000 3.524029e+000 3.336633e+000 4.190800e+000 4.308127e+000 4.198907e+000 4.401058e+000 \
                4.303515e+000 3.945027e+000 3.243079e+000 2.985245e+000 2.440658e+000 5.602147e+000 6.298034e-001 \
                5.834209e-001 5.080934e-001 6.770245e-001 7.052047e-001 7.098504e-001 7.490557e-001 7.594697e-001 \
                6.945505e-001 5.957313e-001 5.417506e-001 4.463942e-001 8.994048e-001',file=fw5)
    print('<GCONST> 1.383837e+002',file=fw5)
    print('<STATE> 3',file=fw5)
    print('<MEAN> 39',file=fw5)
    print(' -1.012254e+001 -7.264226e+000 -5.370609e+000 -9.712223e+000 -8.047407e+000 -5.681805e+000 \
                -5.715530e+000 -8.866713e-001 -3.691204e+000 -1.679087e+000 -2.372116e+000 -1.992831e+000 5.435896e+001 \
                3.500786e-003 3.751630e-003 -3.257445e-004 8.666397e-004 4.723764e-003 -5.870885e-004 2.242961e-004 \
                -5.077728e-003 -4.136859e-003 -6.560616e-004 -4.315113e-003 -2.692402e-003 3.635559e-003 -9.592683e-005 \
                -2.103158e-004 1.351117e-004 -1.661481e-005 -3.384908e-004 -2.177467e-004 -1.438305e-004 1.636888e-004 \
                2.628795e-004 5.652815e-006 2.228666e-004 2.034386e-004 -1.619475e-005',file=fw5)
    print('<VARIANCE> 39',file=fw5)
    print(' 1.019786e+002 5.942492e+001 7.969102e+001 8.208926e+001 7.801923e+001 7.487221e+001 \
            7.644022e+001 6.621936e+001 6.555122e+001 4.695240e+001 4.796211e+001 3.524432e+001 1.175200e+002 \
            4.562054e+000 3.524029e+000 3.336633e+000 4.190800e+000 4.308127e+000 4.198907e+000 4.401058e+000 \
            4.303515e+000 3.945027e+000 3.243079e+000 2.985245e+000 2.440658e+000 5.602147e+000 6.298034e-001 \
            5.834209e-001 5.080934e-001 6.770245e-001 7.052047e-001 7.098504e-001 7.490557e-001 7.594697e-001 \
            6.945505e-001 5.957313e-001 5.417506e-001 4.463942e-001 8.994048e-001',file=fw5)
    print('<GCONST> 1.383837e+002',file=fw5)
    print('<STATE> 4',file=fw5)
    print('<MEAN> 39',file=fw5)
    print(' -1.012254e+001 -7.264226e+000 -5.370609e+000 -9.712223e+000 -8.047407e+000 -5.681805e+000 \
            -5.715530e+000 -8.866713e-001 -3.691204e+000 -1.679087e+000 -2.372116e+000 -1.992831e+000 \
            5.435896e+001 3.500786e-003 3.751630e-003 -3.257445e-004 8.666397e-004 4.723764e-003 -5.870885e-004 \
            2.242961e-004 -5.077728e-003 -4.136859e-003 -6.560616e-004 -4.315113e-003 -2.692402e-003 3.635559e-003 \
            -9.592683e-005 -2.103158e-004 1.351117e-004 -1.661481e-005 -3.384908e-004 -2.177467e-004 -1.438305e-004 \
            1.636888e-004 2.628795e-004 5.652815e-006 2.228666e-004 2.034386e-004 -1.619475e-005',file=fw5)
    print('<VARIANCE> 39',file=fw5)
    print(' 1.019786e+002 5.942492e+001 7.969102e+001 8.208926e+001 7.801923e+001 7.487221e+001 \
                7.644022e+001 6.621936e+001 6.555122e+001 4.695240e+001 4.796211e+001 3.524432e+001 1.175200e+002 \
                4.562054e+000 3.524029e+000 3.336633e+000 4.190800e+000 4.308127e+000 4.198907e+000 4.401058e+000 \
                4.303515e+000 3.945027e+000 3.243079e+000 2.985245e+000 2.440658e+000 5.602147e+000 6.298034e-001 \
                5.834209e-001 5.080934e-001 6.770245e-001 7.052047e-001 7.098504e-001 7.490557e-001 7.594697e-001 \
                6.945505e-001 5.957313e-001 5.417506e-001 4.463942e-001 8.994048e-001',file=fw5)
    print('<GCONST> 1.383837e+002',file=fw5)
    print('<TRANSP> 5',file=fw5)
    print(' 0.000000e+000 1.000000e+000 0.000000e+000 0.000000e+000 0.000000e+000',file=fw5)
    print(' 0.000000e+000 6.000000e-001 4.000000e-001 0.000000e+000 0.000000e+000',file=fw5)
    print(' 0.000000e+000 0.000000e+000 6.000000e-001 4.000000e-001 0.000000e+000',file=fw5)
    print(' 0.000000e+000 0.000000e+000 0.000000e+000 7.000000e-001 3.000000e-001',file=fw5)
    print(' 0.000000e+000 0.000000e+000 0.000000e+000 0.000000e+000 0.000000e+000',file=fw5)
    print('<ENDHMM>',file=fw5)
    fw5.close()
    
    #训练 解码 配置config文件
    fw6 = open('configure/config','w')
    print('TARGETKIND=MFCC_0_D_A',file=fw6)
    print('TARGETRATE=100000.0',file=fw6)
    print('SAVECOMPRESSED=T',file=fw6)
    print('USEHAMMING=T',file=fw6)
    print('WINDOWSIZE=250000.0',file=fw6)
    print ('SAVEWITHCRC=T',file=fw6)
    print('PREEMCOEF=0.97',file=fw6)
    print('NUMCHANS=26',file=fw6)
    print('CEPLIFTER=22',file=fw6)
    print('NUMCEPS=12',file=fw6)
    fw6.close()

In [10]:
def defined_single_model():
    # 初始化单因子模型，定义hmms
    open('hmms/hmm0/macros','w').write('~o \n<VECSIZE> 39 <MFCC_0_D_A>\n' + open('hmms/hmm0/vFloors','r').read())
    phones = open('phones/monophones0','r').read().split('\n')
    # 新生成的proto文件
    proto_txt = open('hmms/hmm0/proto','r').read()
    head_index = proto_txt.find('~o')
    phones_index = proto_txt.find('~h')
    hmm_index = proto_txt.find('<BEGINHMM>')
    head_txt = proto_txt[head_index:phones_index]
    phone_txt = proto_txt[phones_index:hmm_index]
    hmm_txt = proto_txt[hmm_index:]
    models=[phone_txt.replace('proto',i)+hmm_txt for i in phones]
    models=head_txt+''.join(models)
    open('hmms/hmm0/hmmdefs','w').write(models)    
    # 修补静音模式
    if not os.path.exists('hedfile') : os.mkdir('hedfile')
    fw7 = open('hedfile/sil.hed','w')
    print('AT 2 4 0.2 {sil.transP}',file=fw7)
    print('AT 4 2 0.2 {sil.transP}',file=fw7)
    print('AT 1 3 0.3 {sp.transP}',file=fw7)
    print('TI silst {sil.state[3],sp.state[2]}',file=fw7)
    
    # 给dict2添加SIL sil
    open('dict/dict2','w').write('SIL sil\n' + open('dict/dict1','r').read())

In [11]:
def add_sp_model(model_hmmdefs):
    fw = open('/home/godlovefe/CRHTK/hmms/hmm4/hmmdefs_sp','w')
    model_txt = open(model_hmmdefs,'r').read()
    begin = model_txt.find(r'~h "sil"')
    end = model_txt.find('<ENDHMM>',begin)
    sil_model = model_txt[begin:end + 9].replace('sil','sp')
    fw.write(model_txt + sil_model)

In [1]:
def tri_hmm():
    # 由单音素得到三音素并重估参数
    fw = open('ledfile/mktri.led','w').write('WB sp\nWB sil\nTC')
    cmd1 = 'HLEd -n phones/triphones1 -l "*" -i labels/wintri.mlf ledfile/mktri.led labels/aligned.mlf'
    os.system(cmd1)
    
    # 生成mktri.hed文件
    cmd2 = 'perl HTKTutorial/maketrihed phones/monophones1 phones/triphones1'
    os.system(cmd2)
    
    #初始化三因素模型
    if not os.path.exists('hmms/hmm9') : os.mkdir('hmms/hmm9')
        if not os.path.exists('hedfile') : os.mkdir('hedfile')
    cmd3 = 'HHEd -H hmms/hmm9/macros -H hmms/hmm9/hmmdefs_sp -M hmms/hmm10 hedfile/mktri.hed phones/monophones1'
    os.system(cmd3)
    
    # 重估两次
    # time 1
    if not os.path.exists('hmms/hmm11') : os.mkdir('hmms/hmm11')
    cmd4_1 = 'HERest -T 1 -C configure/config -I labels/wintri.mlf -t 250.0 150.0 1000.0 -s stats \
             -S scpfile/train1.scp -H hmms/hmm10/hmmdefs_sp -H hmms/hmm10/macros -M hmms/hmm11 phones/triphones1'
    os.system(cmd4_1)
    # time 2
    if not os.path.exists('hmms/hmm12') : os.mkdir('hmms/hmm12')
    cmd4_2 = 'HERest -T 1 -C configure/config -I labels/wintri.mlf -t 250.0 150.0 1000.0 -s stats \
              -S scpfile/train1.scp  -H hmms/hmm11/hmmdefs_sp -H hmms/hmm11/macros -M hmms/hmm12 phones/triphones1'
    os.system(cmd4_2)
    
#     # 注释字典中发音
#     cmd_dict = 'perl HTKTutorial/makedict.pl dict/dict2 dict/dict2_1 phones/triphones1'
#     os.system(cmd_dict)
    
    # result
    if not os.path.exists('results') : os.mkdir('results')
    cmd_7 = 'HVite -H hmms/hmm12/macros -H hmms/hmm12/hmmdefs_sp -S scpfile/test.scp -l "*" -i \
             results/result_hmm12.mlf -w wdnet/wdnet -p 0.0 -s 5.0 dict/dict2_1 phones/triphones1'
    os.system(cmd_7)

    cmd = 'HResults -I labels/testwords.mlf phones/monophones1 results/result_hmm12.mlf'
    os.system(cmd)
    
    # 制作fulllist
    cmd = 'HDMan -b sp -n list/fulllist -g dedfile/global.ded -l log/flog dict/dict3_tri dict/dict3'
    #生成tree.hed文件
    cmd = 'perl HTKTutorial/mkclscript.prl TB 350.0 phones/monophones1 >hedfile/tree.hed'
    os.system(cmd)
    # 绑定三因素
    if not os.path.exists('hmms/hmm13') : os.mkdir('hmms/hmm13')
    cmd5 = 'HHEd -T 1 -B -H hmms/hmm12/macros -H hmms/hmm12/hmmdefs_sp -M hmms/hmm13 hedfile/tree.hed \
             phones/triphones1 >log/Triphoneslog'
    os.system(cmd5)
    
    cmd6 = 'HERest -T 1 -C configure/config -I labels/wintri.mlf -t 250.0 150.0 1000.0 -S scpfile/train1.scp \
            -H hmms/hmm13/macros -H hmms/hmm13/hmmdefs_sp -M hmms/hmm14 list/tiedlist'
    os.system(cmd6)
 
    cmd7 = 'HERest -T 1 -C configure/config -I labels/wintri.mlf -t 250.0 150.0 1000.0 -S scpfile/train1.scp \
            -H hmms/hmm14/macros -H hmms/hmm14/hmmdefs_sp -M hmms/hmm15 list/tiedlist'
    os.system(cmd7)
    
    # 识别率验证
    cmd_7 = 'HVite -T 1 -H hmms/hmm15/macros -H hmms/hmm15/hmmdefs_sp -S scpfile/test.scp -l "*" -i \
             results/result_hmm15.mlf -w wdnet/wdnet -p 0.0 -s 5.0 dict/dict2_1 list/tiedlist'
    os.system(cmd_7)

    cmd = 'HResults -e ??? SENT-START -e ??? SENT-END -I labels/testwords.mlf list/tiedlist results/result_hmm15.mlf'
    os.system(cmd)


In [ ]:
#基础数据准备
# prepare_basic_data(thchs30)
#数据准备
# prepare_train_data()
# 特征提取
# feature_extraction(thchs30)
# cmd = 'HCopy -T 1 -C configure/wav_config -S scpfile/trainmfcc.scp'
# os.system(cmd)
# cmd = 'HCopy -T 1 -C configure/wav_config -S scpfile/testmfcc.scp'
# os.system(cmd)

In [ ]:
# #统计均值以及方差
# calculate_mean_variance()
# if not os.path.exists('hmms') : os.mkdir('hmms')
# if not os.path.exists('hmms/hmm0') : os.mkdir('hmms/hmm0')
# cmd = 'HCompV -T 1 -C configure/config -f 0.01 -m -S scpfile/train.scp -M hmms/hmm0 proto'
# os.system(cmd)  

In [ ]:
#单因素模型及评估
defined_single_model()
#重估三次
# time 1
if not os.path.exists('hmms/hmm1') : os.mkdir('hmms/hmm1')
cmd1 = 'HERest -C configure/config -I labels/trainphones0.mlf -t 250.0 150.0 1000.0 \
          -S scpfile/train.scp -H hmms/hmm0/macros -H hmms/hmm0/hmmdefs -M hmms/hmm1 phones/monophones0'
os.system(cmd1)
# time 2
if not os.path.exists('hmms/hmm2') : os.mkdir('hmms/hmm2')
cmd2 = 'HERest -T 1 -C configure/config -I labels/trainphones0.mlf -t 250.0 150.0 1000.0 \
          -S scpfile/train.scp -H hmms/hmm1/macros -H hmms/hmm1/hmmdefs -M hmms/hmm2 phones/monophones0'
os.system(cmd2)
# time 3
if not os.path.exists('hmms/hmm3') : os.mkdir('hmms/hmm3')
cmd3 = 'HERest  -C configure/config -I labels/trainphones0.mlf -t 250.0 150.0 1000.0 \
          -S scpfile/train.scp -H hmms/hmm2/macros -H hmms/hmm2/hmmdefs -M hmms/hmm3 phones/monophones0'
os.system(cmd3)


# 将hmm3中的hmmdefs文件的”sil”模型复制，并改为“sp”，然后把新的hmmdefs跟macros一起放在hmm4文件夹中
# 修补静音模式
add_sp_model('/home/godlovefe/CRHTK/hmms/hmm4/hmmdefs')
if not os.path.exists('hmms/hmm5') : os.mkdir('hmms/hmm5')
cmd4 = 'HHEd -A -D -T  1 -H hmms/hmm4/macros -H hmms/hmm4/hmmdefs_sp -M hmms/hmm5 hedfile/sil.hed phones/monophones1'
os.system(cmd4)

# 修改mkphones0.led,去掉最后一行，存为mkphones1.led，利用HLEd工具得到包含sp的音素级真值文本：
# 包含sp的音素级真值文本
cmd_sp = 'HLEd -l "*" -d dict/dict1 -i labels/trainphones1.mlf ledfile/mkphones1.led  labels/trainwords.mlf '
os.system(cmd_sp)

#重估两次
# time 1
if not os.path.exists('hmms/hmm6') : os.mkdir('hmms/hmm6')
cmd5 = 'HERest -C configure/config -I labels/trainphones1.mlf -t 250.0 150.0 1000.0 \
          -S scpfile/train.scp -H hmms/hmm5/macros -H hmms/hmm5/hmmdefs_sp -M hmms/hmm6 phones/monophones1'
os.system(cmd5)
# time 2
if not os.path.exists('hmms/hmm7') : os.mkdir('hmms/hmm7')
cmd6 = 'HERest -C configure/config -I labels/trainphones1.mlf -t 250.0 150.0 1000.0 \
          -S scpfile/train.scp -H hmms/hmm6/macros -H hmms/hmm6/hmmdefs_sp -M hmms/hmm7 phones/monophones1'
os.system(cmd6)
# result
if not os.path.exists('results') : os.mkdir('results')
cmd_7 = 'HVite -H hmms/hmm7/macros -H hmms/hmm7/hmmdefs_sp -S scpfile/test.scp -l "*" -i results/result_hmm7.mlf \
         -w wdnet/wdnet -p 0.0 -s 5.0 dict/dict1_1 phones/monophones1'
os.system(cmd_7)

#SENT: %Correct=0.00 [H=0, S=2495, N=2495]
#WORD: %Corr=8.09, Acc=-7.26 [H=3972, D=1440, S=43673, I=7537, N=49085]
cmd = 'HResults -I labels/testwords.mlf phones/monophones1 results/recout_step7.mlf'
os.system(cmd)

# 重新调整训练数据
cmd7 = 'HVite -T 1 -l "*" -o SWT -b SIL -C configure/config -a -H hmms/hmm7/macros -H hmms/hmm7/hmmdefs_sp \
        -i labels/aligned.mlf -m -t 250.0 -y lab -I labels/trainwords.mlf -S scpfile/train.scp dict/dict2  \
        phones/monophones1'
os.system(cmd7)

#删除aligned之后的训练样本
train_scp=open('data/train/train.scp','r').read()
list=train_scp.split('\n')
search=open('data/train/aligned.mlf','r').read()
count=0
if os.path.exists('data/train/train1.scp') : os.remove('data/train/train1.scp')
fw=open('data/train/train1.scp','a')
for i in list:
    key=i.split('/')[-1][:-4]+'.lab'
    yn=search.find(key)
    if yn==-1:
        count+=1
    else:
        fw.write(i+'\n')
fw.close()

# 再次重估两次
# time 1
if not os.path.exists('hmms/hmm8') : os.mkdir('hmms/hmm8')
cmd8 = 'HERest -T 1 -C configure/config -I labels/aligned.mlf -t 250.0 150.0 1000.0 \
          -S scpfile/train1.scp -H hmms/hmm7/macros -H hmms/hmm7/hmmdefs_sp -M hmms/hmm8 phones/monophones1'
os.system(cmd8)
# time 2
if not os.path.exists('hmms/hmm9') : os.mkdir('hmms/hmm9')
cmd9 = 'HERest -T  1 -C configure/config -I labels/trainphones1.mlf -t 250.0 150.0 1000.0 \
          -S scpfile/train1.scp -H hmms/hmm8/macros -H hmms/hmm8/hmmdefs_sp -M hmms/hmm9 phones/monophones1'
os.system(cmd9)

# result
if not os.path.exists('results') : os.mkdir('results')
cmd10 = 'HVite -H hmms/hmm9/macros -H hmms/hmm9/hmmdefs_sp -S scpfile/test.scp  -l "*" -i results/result_hmm9.mlf \
         -w wdnet/wdnet -p 0.0 -s 5.0 dict/dict2 phones/monophones1'
os.system(cmd10)

# 测试正确率
# SENT: %Correct=0.00 [H=0, S=2495, N=2495]
# WORD: %Corr=8.93, Acc=-11.12 [H=4381, D=959, S=43745, I=9837, N=49085]
cmd11 = 'HResults -f -I labels/testwords.mlf phones/monophones1 results/result_hmm9.mlf \
         >/home/godlovefe/CRHTK/results/result_hmm9.txt'
os.system(cmd11)